# This EDA tend to prepare our dataset to be processed by our algorithms.

We want the dataset to have relevant data so we will exclude everything that doesn't contains informations about agriculture or machine learning, we will discard everything that is not obviouslt english, weirds ponctuation and everything we judge bad for our algorithm performances.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_json('/content/gdrive/MyDrive/Exam/AgrSmall.json')
df.head()

,doi,titles,abstracts,authors,keywords,sources
0,10.3390,Community Faecal Management Strategies and Per...,Most people in rural areas in South Africa (SA...,"[Matthew Mamera, Johan J. van Tol, Makhosazana...",agriculture,mdpi
1,10.3390,Adoption of Sustainable Agriculture Practices ...,The aim of this study was to highlight the imp...,"[Rafay Waseem, Gershom Endelani Mwalupaso, Far...",agriculture,mdpi
2,10.3390,Atlanta Residents’ Knowledge Regarding Heavy M...,Urban agriculture and gardening provide many h...,"[Lauren Balotin, Samantha Distler, Antoinette ...",agriculture,mdpi
3,10.3390,Perceptions of the Challenges and Opportunitie...,Waste management has become pertinent in urban...,"[Nqubeko Neville Menyuka, Melusi Sibanda, Urmi...",agriculture,mdpi
4,10.3390,An Assessment of Seaweed Extracts: Innovation ...,Plant growth regulators (PGRs) are described i...,"[El Chami Daniel, Galli Fabio]",agriculture,mdpi


In [ ]:
df.describe()

,doi,titles,abstracts,authors,keywords,sources
count,4531,4531,4531,4531,4531,4531
unique,169,4011,3879,3761,33,419
top,10.1016/,ombining computer vision and deep learning to ...,No abstract is available for this item.,[],agronomy,arxiv
freq,1293,5,16,145,321,652


In [ ]:
docs = list(df.loc[:, "abstracts"].values)

In [ ]:
docs

['Most people in rural areas in South Africa (SA) rely on untreated drinking groundwater sources and pit latrine sanitations. A minimum basic sanitation facility should enable safe and appropriate removal of human waste, and although pit latrines provide this, they are still contamination concerns. Pit latrine sludge in SA is mostly emptied and disposed off-site as waste or buried in-situ. Despite having knowledge of potential sludge benefits, most communities in SA are reluctant to use it. This research captured social perceptions regarding latrine sludge management in Monontsha village in the Free State Province of SA through key informant interviews and questionnaires. A key informant interview and questionnaire was done in Monontsha, SA. Eighty participants, representing 5% of all households, were selected. Water samples from four boreholes and four rivers were analyzed for faecal coliforms and E.coli bacteria. On average, five people in a household were sharing a pit latrine. Eigh

Clearly some text have a very bad beggining like @@@@Highlights • 

# Ponctuation

In [ ]:
import re
import string
print(string.punctuation)


!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
def remove_punctuation(text):
    text = ''.join([char for char in text if char not in string.punctuation])
    # remove numerical values
    text = re.sub(r'\d+', '', text)
    # substitute multiple whitespace with single whitespace and remove leading and trailing whitespaces 
    text = re.sub('\s+', ' ', text).strip() 
    return text

In [ ]:
df['text_clean'] = df['abstracts'].apply(lambda x: remove_punctuation(x))
df

,doi,titles,abstracts,authors,keywords,sources,text_clean
0,10.3390,Community Faecal Management Strategies and Per...,Most people in rural areas in South Africa (SA...,"[Matthew Mamera, Johan J. van Tol, Makhosazana...",agriculture,mdpi,Most people in rural areas in South Africa SA ...
1,10.3390,Adoption of Sustainable Agriculture Practices ...,The aim of this study was to highlight the imp...,"[Rafay Waseem, Gershom Endelani Mwalupaso, Far...",agriculture,mdpi,The aim of this study was to highlight the imp...
2,10.3390,Atlanta Residents’ Knowledge Regarding Heavy M...,Urban agriculture and gardening provide many h...,"[Lauren Balotin, Samantha Distler, Antoinette ...",agriculture,mdpi,Urban agriculture and gardening provide many h...
3,10.3390,Perceptions of the Challenges and Opportunitie...,Waste management has become pertinent in urban...,"[Nqubeko Neville Menyuka, Melusi Sibanda, Urmi...",agriculture,mdpi,Waste management has become pertinent in urban...
4,10.3390,An Assessment of Seaweed Extracts: Innovation ...,Plant growth regulators (PGRs) are described i...,"[El Chami Daniel, Galli Fabio]",agriculture,mdpi,Plant growth regulators PGRs are described in ...
...,...,...,...,...,...,...,...
4526,10.2478/,1. Modelling groundwater flow and nitrate tran...,The present paper discusses studies related to...,"[Sieczka Anna, Bujakowski Filip, Koda Eugeniusz]",precision agriculture,"Sciendo, 2018.",The present paper discusses studies related to...
4527,unknown,2. Cosechando los beneficios de la agricultura...,El objetivo del trabajo fue desarrollar una me...,"[Bonilla, Camila, Terra, José A, Gutiérrez, Lu...",precision agriculture,Facultad de Agronomía - Instituto Nacional de ...,El objetivo del trabajo fue desarrollar una me...
4528,unknown,6. A Risk Analysis of Precision Agriculture Te...,Precision agriculture technology can transform...,"[Yangxuan Liu, Michael R. Langemeier, Ian M. S...",precision agriculture,"MDPI, Open Access Journal, 2018.",Precision agriculture technology can transform...
4529,10.1016/,7. Integrated open geospatial web service enab...,Highlights •We proposed an integrated geospati...,"[Chen, Nengcheng, Zhang, Xiang, Wang, Chao]",precision agriculture,Elsevier B.V.,Highlights •We proposed an integrated geospati...


# Obvious words that should not be here

In [ ]:
stop_words = ["Highlights", "Resumen", "Abstract", "Introduction", "©",'Keywords','•','INTRODUCTION','Background']


In [ ]:
print(stop_words)

['Highlights', 'Resumen', 'Abstract', 'Introduction', '©', 'Keywords', '•', 'INTRODUCTION', 'Background']


In [ ]:

def remove_stopwords(x):
  words = x.split(" ")
  temp = []
  for word in words:
    if word not in stop_words:
      temp.append(word)
  return " ".join(temp)

In [ ]:
df['text_clean'] = df['text_clean'].apply(lambda x: remove_stopwords(x))
df

,doi,titles,abstracts,authors,keywords,sources,text_clean
0,10.3390,Community Faecal Management Strategies and Per...,Most people in rural areas in South Africa (SA...,"[Matthew Mamera, Johan J. van Tol, Makhosazana...",agriculture,mdpi,Most people in rural areas in South Africa SA ...
1,10.3390,Adoption of Sustainable Agriculture Practices ...,The aim of this study was to highlight the imp...,"[Rafay Waseem, Gershom Endelani Mwalupaso, Far...",agriculture,mdpi,The aim of this study was to highlight the imp...
2,10.3390,Atlanta Residents’ Knowledge Regarding Heavy M...,Urban agriculture and gardening provide many h...,"[Lauren Balotin, Samantha Distler, Antoinette ...",agriculture,mdpi,Urban agriculture and gardening provide many h...
3,10.3390,Perceptions of the Challenges and Opportunitie...,Waste management has become pertinent in urban...,"[Nqubeko Neville Menyuka, Melusi Sibanda, Urmi...",agriculture,mdpi,Waste management has become pertinent in urban...
4,10.3390,An Assessment of Seaweed Extracts: Innovation ...,Plant growth regulators (PGRs) are described i...,"[El Chami Daniel, Galli Fabio]",agriculture,mdpi,Plant growth regulators PGRs are described in ...
...,...,...,...,...,...,...,...
4526,10.2478/,1. Modelling groundwater flow and nitrate tran...,The present paper discusses studies related to...,"[Sieczka Anna, Bujakowski Filip, Koda Eugeniusz]",precision agriculture,"Sciendo, 2018.",The present paper discusses studies related to...
4527,unknown,2. Cosechando los beneficios de la agricultura...,El objetivo del trabajo fue desarrollar una me...,"[Bonilla, Camila, Terra, José A, Gutiérrez, Lu...",precision agriculture,Facultad de Agronomía - Instituto Nacional de ...,El objetivo del trabajo fue desarrollar una me...
4528,unknown,6. A Risk Analysis of Precision Agriculture Te...,Precision agriculture technology can transform...,"[Yangxuan Liu, Michael R. Langemeier, Ian M. S...",precision agriculture,"MDPI, Open Access Journal, 2018.",Precision agriculture technology can transform...
4529,10.1016/,7. Integrated open geospatial web service enab...,Highlights •We proposed an integrated geospati...,"[Chen, Nengcheng, Zhang, Xiang, Wang, Chao]",precision agriculture,Elsevier B.V.,•We proposed an integrated geospatial service ...


In [ ]:
df = df[~df['text_clean'].astype(str).str.startswith('Note In lieu')]

## Russian rows

In [ ]:
# function to remove non-ASCII 
def remove_non_ascii(text): 
    return ''.join(i for i in text if ord(i)<128) 
 
df['text_clean'] = df['text_clean'].apply(remove_non_ascii) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Here we take out the russian rows

In [ ]:
df =df[df['text_clean'].map(len) > 50]

In [ ]:
df = df[~df['text_clean'].astype(str).str.startswith('   ')]

These are deep adjustement as i read the dataset rapidly. Sentences that begins with that only indicate technical consideration like 'this paper is edited this date etccc' 

In [ ]:
df = df[~df['text_clean'].astype(str).str.startswith('The authors')]

Suppression of duplicate

In [ ]:
df.drop_duplicates(subset=['text_clean'], keep='last')

,doi,titles,abstracts,authors,keywords,sources,text_clean
0,10.3390,Community Faecal Management Strategies and Per...,Most people in rural areas in South Africa (SA...,"[Matthew Mamera, Johan J. van Tol, Makhosazana...",agriculture,mdpi,Most people in rural areas in South Africa SA ...
1,10.3390,Adoption of Sustainable Agriculture Practices ...,The aim of this study was to highlight the imp...,"[Rafay Waseem, Gershom Endelani Mwalupaso, Far...",agriculture,mdpi,The aim of this study was to highlight the imp...
2,10.3390,Atlanta Residents’ Knowledge Regarding Heavy M...,Urban agriculture and gardening provide many h...,"[Lauren Balotin, Samantha Distler, Antoinette ...",agriculture,mdpi,Urban agriculture and gardening provide many h...
3,10.3390,Perceptions of the Challenges and Opportunitie...,Waste management has become pertinent in urban...,"[Nqubeko Neville Menyuka, Melusi Sibanda, Urmi...",agriculture,mdpi,Waste management has become pertinent in urban...
4,10.3390,An Assessment of Seaweed Extracts: Innovation ...,Plant growth regulators (PGRs) are described i...,"[El Chami Daniel, Galli Fabio]",agriculture,mdpi,Plant growth regulators PGRs are described in ...
...,...,...,...,...,...,...,...
4526,10.2478/,1. Modelling groundwater flow and nitrate tran...,The present paper discusses studies related to...,"[Sieczka Anna, Bujakowski Filip, Koda Eugeniusz]",precision agriculture,"Sciendo, 2018.",The present paper discusses studies related to...
4527,unknown,2. Cosechando los beneficios de la agricultura...,El objetivo del trabajo fue desarrollar una me...,"[Bonilla, Camila, Terra, José A, Gutiérrez, Lu...",precision agriculture,Facultad de Agronomía - Instituto Nacional de ...,El objetivo del trabajo fue desarrollar una me...
4528,unknown,6. A Risk Analysis of Precision Agriculture Te...,Precision agriculture technology can transform...,"[Yangxuan Liu, Michael R. Langemeier, Ian M. S...",precision agriculture,"MDPI, Open Access Journal, 2018.",Precision agriculture technology can transform...
4529,10.1016/,7. Integrated open geospatial web service enab...,Highlights •We proposed an integrated geospati...,"[Chen, Nengcheng, Zhang, Xiang, Wang, Chao]",precision agriculture,Elsevier B.V.,We proposed an integrated geospatial service e...


In [ ]:
docs = list(df.loc[:, "text_clean"].values)

# Result

In [ ]:
docs

['Most people in rural areas in South Africa SA rely on untreated drinking groundwater sources and pit latrine sanitations A minimum basic sanitation facility should enable safe and appropriate removal of human waste and although pit latrines provide this they are still contamination concerns Pit latrine sludge in SA is mostly emptied and disposed offsite as waste or buried insitu Despite having knowledge of potential sludge benefits most communities in SA are reluctant to use it This research captured social perceptions regarding latrine sludge management in Monontsha village in the Free State Province of SA through key informant interviews and questionnaires A key informant interview and questionnaire was done in Monontsha SA Eighty participants representing of all households were selected Water samples from four boreholes and four rivers were analyzed for faecal coliforms and Ecoli bacteria On average five people in a household were sharing a pit latrine Eightythree percent disposed

#Exportation

In [ ]:
df['abstracts']=df['text_clean']
df = df.drop(['text_clean'], axis=1)
df

,doi,titles,abstracts,authors,keywords,sources
0,10.3390,Community Faecal Management Strategies and Per...,Most people in rural areas in South Africa SA ...,"[Matthew Mamera, Johan J. van Tol, Makhosazana...",agriculture,mdpi
1,10.3390,Adoption of Sustainable Agriculture Practices ...,The aim of this study was to highlight the imp...,"[Rafay Waseem, Gershom Endelani Mwalupaso, Far...",agriculture,mdpi
2,10.3390,Atlanta Residents’ Knowledge Regarding Heavy M...,Urban agriculture and gardening provide many h...,"[Lauren Balotin, Samantha Distler, Antoinette ...",agriculture,mdpi
3,10.3390,Perceptions of the Challenges and Opportunitie...,Waste management has become pertinent in urban...,"[Nqubeko Neville Menyuka, Melusi Sibanda, Urmi...",agriculture,mdpi
4,10.3390,An Assessment of Seaweed Extracts: Innovation ...,Plant growth regulators PGRs are described in ...,"[El Chami Daniel, Galli Fabio]",agriculture,mdpi
...,...,...,...,...,...,...
4526,10.2478/,1. Modelling groundwater flow and nitrate tran...,The present paper discusses studies related to...,"[Sieczka Anna, Bujakowski Filip, Koda Eugeniusz]",precision agriculture,"Sciendo, 2018."
4527,unknown,2. Cosechando los beneficios de la agricultura...,El objetivo del trabajo fue desarrollar una me...,"[Bonilla, Camila, Terra, José A, Gutiérrez, Lu...",precision agriculture,Facultad de Agronomía - Instituto Nacional de ...
4528,unknown,6. A Risk Analysis of Precision Agriculture Te...,Precision agriculture technology can transform...,"[Yangxuan Liu, Michael R. Langemeier, Ian M. S...",precision agriculture,"MDPI, Open Access Journal, 2018."
4529,10.1016/,7. Integrated open geospatial web service enab...,We proposed an integrated geospatial service e...,"[Chen, Nengcheng, Zhang, Xiang, Wang, Chao]",precision agriculture,Elsevier B.V.


In [ ]:
df.to_json(path_or_buf="/content/gdrive/MyDrive/Exam/AgrSmall_cleaned.json")

In [ ]:
import pandas as pd
df = pd.read_json('/content/gdrive/MyDrive/Exam/AgrSmall_cleaned.json')
df.head()

,doi,titles,abstracts,authors,keywords,sources
0,10.3390,Community Faecal Management Strategies and Per...,Most people in rural areas in South Africa SA ...,"[Matthew Mamera, Johan J. van Tol, Makhosazana...",agriculture,mdpi
1,10.3390,Adoption of Sustainable Agriculture Practices ...,The aim of this study was to highlight the imp...,"[Rafay Waseem, Gershom Endelani Mwalupaso, Far...",agriculture,mdpi
2,10.3390,Atlanta Residents’ Knowledge Regarding Heavy M...,Urban agriculture and gardening provide many h...,"[Lauren Balotin, Samantha Distler, Antoinette ...",agriculture,mdpi
3,10.3390,Perceptions of the Challenges and Opportunitie...,Waste management has become pertinent in urban...,"[Nqubeko Neville Menyuka, Melusi Sibanda, Urmi...",agriculture,mdpi
4,10.3390,An Assessment of Seaweed Extracts: Innovation ...,Plant growth regulators PGRs are described in ...,"[El Chami Daniel, Galli Fabio]",agriculture,mdpi


In [ ]:
docs = list(df.loc[:, "abstracts"].values)

In [387]:
docs

['Most people in rural areas in South Africa SA rely on untreated drinking groundwater sources and pit latrine sanitations A minimum basic sanitation facility should enable safe and appropriate removal of human waste and although pit latrines provide this they are still contamination concerns Pit latrine sludge in SA is mostly emptied and disposed offsite as waste or buried insitu Despite having knowledge of potential sludge benefits most communities in SA are reluctant to use it This research captured social perceptions regarding latrine sludge management in Monontsha village in the Free State Province of SA through key informant interviews and questionnaires A key informant interview and questionnaire was done in Monontsha SA Eighty participants representing of all households were selected Water samples from four boreholes and four rivers were analyzed for faecal coliforms and Ecoli bacteria On average five people in a household were sharing a pit latrine Eightythree percent disposed